# multimodal input processing

### optical character recognition (OCR)

In [42]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

!pip install easyocr

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 908.3/908.3 kB 49.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 31.6 MB/s eta 0:00:00


In [43]:
import easyocr
reader = easyocr.Reader(['en']) # english

Progress: |██████████████████████████████████████████████████| 100.0% Complete

Progress: |██████████████████████████████████████████████████| 100.0% Complete

In [44]:
extracted = reader.readtext('img.png')

In [45]:
extract = ""

for (bbox, text, prob) in extracted:
    extract = extract + text
    extract = extract + " "

In [46]:
extract

'"Our greatest weakness lies in giving up: The most certain way to succeed is always to try just one more time:\' Thomas Edison Forbes '

### image captioning

In [47]:
!pip install keras_ocr

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 916.6 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 31.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 7.4 MB/s eta 0:00:00


In [48]:
import matplotlib.pyplot as plt
import keras_ocr
import cv2
import math
import numpy as np

In [55]:
def midpoint(x1, y1, x2, y2):
    x_mid = int((x1 + x2)/2)
    y_mid = int((y1 + y2)/2)
    return (x_mid, y_mid)

In [56]:
def inpaint_text(img_path, pipeline):
    img = keras_ocr.tools.read(img_path)
    prediction_groups = pipeline.recognize([img])

    mask = np.zeros(img.shape[:2], dtype="uint8")
    for box in prediction_groups[0]:
        x0, y0 = box[1][0]
        x1, y1 = box[1][1]
        x2, y2 = box[1][2]
        x3, y3 = box[1][3]

        x_mid0, y_mid0 = midpoint(x1, y1, x2, y2)
        x_mid1, y_mid1 = midpoint(x0, y0, x3, y3)

        thickness = int(math.sqrt( (x2 - x1)**2 + (y2 - y1)**2 ))

        cv2.line(mask, (x_mid0, y_mid0), (x_mid1, y_mid1), 255,
        thickness)
        inpainted_img = cv2.inpaint(img, mask, 7, cv2.INPAINT_NS)

    return(inpainted_img)

In [ ]:
pipeline = keras_ocr.pipeline.Pipeline()

image_textless = inpaint_text('img.png', pipeline)

In [61]:
from transformers import VisionEncoderDecoderModel, ViTFeatureExtractor, AutoTokenizer
import torch
from PIL import Image

In [62]:
model_VED = VisionEncoderDecoderModel.from_pretrained("nlpconnect/vit-gpt2-image-captioning")
feature_extractor = ViTFeatureExtractor.from_pretrained("nlpconnect/vit-gpt2-image-captioning")
tokenizer = AutoTokenizer.from_pretrained("nlpconnect/vit-gpt2-image-captioning")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_VED.to(device)

config.json:   0%|          | 0.00/4.61k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/982M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/228 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/241 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/120 [00:00<?, ?B/s]

VisionEncoderDecoderModel(
  (encoder): ViTModel(
    (embeddings): ViTEmbeddings(
      (patch_embeddings): ViTPatchEmbeddings(
        (projection): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
      )
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): ViTEncoder(
      (layer): ModuleList(
        (0-11): 12 x ViTLayer(
          (attention): ViTAttention(
            (attention): ViTSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (output): ViTSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (intermediate): ViTIntermediate(
            (dense): Linear(in_featur

In [63]:
max_length = 16
num_beams = 4
gen_kwargs = {"max_length": max_length, "num_beams": num_beams}

In [71]:
def predict_step(images):
    pixel_values = feature_extractor(images=images, return_tensors="pt").pixel_values
    pixel_values = pixel_values.to(device)

    output_ids = model_VED.generate(pixel_values, **gen_kwargs)

    preds = tokenizer.batch_decode(output_ids, skip_special_tokens=True)
    preds = [pred.strip() for pred in preds]
    return preds

In [72]:
image = Image.open("/content/img.png")
caption = predict_step(image)

We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.
You may ignore this warning if your `pad_token_id` (50256) is identical to the `bos_token_id` (50256), `eos_token_id` (50256), or the `sep_token_id` (None), and your input is not padded.


In [73]:
caption

['a man in a suit is reading a book']

In [74]:
text = "This guy used to rule America in the 1920s"

In [75]:
multimodal = text + " " + caption[0] + " " + extract

In [76]:
multimodal

'This guy used to rule America in the 1920s a man in a suit is reading a book "Our greatest weakness lies in giving up: The most certain way to succeed is always to try just one more time:\' Thomas Edison Forbes '

# NER using BERT

In [2]:
!pip install simpletransformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.5/315.5 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 5.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 33.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 47.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 27.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.1/267.1 kB 33.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 26.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 46.

In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from simpletransformers.ner import NERModel,NERArgs

In [2]:
data = pd.read_csv("ner_dataset.csv", encoding="latin1")
data = data.fillna(method ="ffill")

In [3]:
data.head(54)

,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,Sentence: 1,of,IN,O
2,Sentence: 1,demonstrators,NNS,O
3,Sentence: 1,have,VBP,O
4,Sentence: 1,marched,VBN,O
5,Sentence: 1,through,IN,O
6,Sentence: 1,London,NNP,B-geo
7,Sentence: 1,to,TO,O
8,Sentence: 1,protest,VB,O
9,Sentence: 1,the,DT,O


In [4]:
data["Sentence #"] = LabelEncoder().fit_transform(data["Sentence #"] )

In [5]:
data.head(54)

,Sentence #,Word,POS,Tag
0,0,Thousands,NNS,O
1,0,of,IN,O
2,0,demonstrators,NNS,O
3,0,have,VBP,O
4,0,marched,VBN,O
5,0,through,IN,O
6,0,London,NNP,B-geo
7,0,to,TO,O
8,0,protest,VB,O
9,0,the,DT,O


In [6]:
data.rename(columns={"Sentence #":"sentence_id","Word":"words","Tag":"labels"}, inplace =True)

In [7]:
data["labels"] = data["labels"].str.upper()

In [8]:
X = data[["sentence_id","words"]]
Y = data["labels"]

In [9]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)

In [10]:
train_data = pd.DataFrame({"sentence_id":x_train["sentence_id"],"words":x_train["words"],"labels":y_train})
test_data = pd.DataFrame({"sentence_id":x_test["sentence_id"],"words":x_test["words"],"labels":y_test})

In [11]:
label = data["labels"].unique().tolist()
label

['O',
 'B-GEO',
 'B-GPE',
 'B-PER',
 'I-GEO',
 'B-ORG',
 'I-ORG',
 'B-TIM',
 'B-ART',
 'I-ART',
 'I-PER',
 'I-GPE',
 'I-TIM',
 'B-NAT',
 'B-EVE',
 'I-EVE',
 'I-NAT']

In [12]:
args = NERArgs()
args.num_train_epochs = 1
args.learning_rate = 1e-4
args.overwrite_output_dir = True
args.train_batch_size = 32
args.eval_batch_size = 32

In [ ]:
model_bert = NERModel('bert', 'bert-base-cased', labels=label, args=args)

model.safetensors: 100%|██████████| 436M/436M [00:11<00:00, 36.7MB/s] 
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
tokenizer_config.json: 100%|██████████| 49.0/49.0 [00:00<?, ?B/s]
vocab.txt: 100%|██████████| 213k/213k [00:00<00:00, 527kB/s]
tokenizer.json: 100%|██████████| 436k/436k [00:00<00:00, 712kB/s]


In [ ]:
model_bert.train_model(train_data, eval_data = test_data, acc=accuracy_score)

Epoch 1 of 1: 100%|██████████| 1/1 [11:11<00:00, 671.75s/it]


(1499, 0.1918389263592059)

In [ ]:
result, model_outputs, preds_list = model_bert.eval_model(test_data)
result

Running Evaluation: 100%|██████████| 1461/1461 [05:16<00:00,  4.62it/s]


{'eval_loss': 0.17083822267256232,
 'precision': 0.8248486516235554,
 'recall': 0.7668937286130033,
 'f1_score': 0.7948161283372831}

In [ ]:
prediction, model_output = model_bert.predict([multimodal])

In [ ]:
prediction

[[{'This': 'O'},
  {'guy': 'O'},
  {'used': 'O'},
  {'to': 'O'},
  {'rule': 'O'},
  {'America': 'B-GEO'},
  {'in': 'O'},
  {'the': 'O'},
  {'1920s': 'B-TIM'},
  {'a': 'O'},
  {'man': 'O'},
  {'wearing': 'O'},
  {'a': 'O'},
  {'suit': 'O'},
  {'and': 'O'},
  {'tie': 'O'},
  {'"Our': 'O'},
  {'greatest': 'O'},
  {'weakness': 'O'},
  {'lies': 'O'},
  {'in': 'O'},
  {'giving': 'O'},
  {'up:': 'O'},
  {'The': 'O'},
  {'most': 'O'},
  {'certain': 'O'},
  {'way': 'O'},
  {'to': 'O'},
  {'succeed': 'O'},
  {'is': 'O'},
  {'always': 'O'},
  {'to': 'O'},
  {'try': 'O'},
  {'just': 'O'},
  {'one': 'O'},
  {'more': 'O'},
  {"time:'": 'O'},
  {'Thomas': 'B-PER'},
  {'Edison': 'I-PER'},
  {'Forbes': 'I-PER'}]]

# NER using LLM (custom entities)

In [15]:
!pip install sentencepiece

In [27]:
!pip install accelerate
# restart kernel

In [17]:
!pip install langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.7/817.7 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.1/289.1 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.7/113.7 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 2.0 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
      Successfully uninstalled packaging-24.0


In [16]:
from langchain import HuggingFacePipeline, PromptTemplate, LLMChain
from transformers import AutoTokenizer, AutoModelForCausalLM
import transformers
import torch

In [14]:
model_LLM = "Universal-NER/UniNER-7B-all"

tokenizer = AutoTokenizer.from_pretrained(model_LLM)
pipeline = transformers.pipeline(
    "text-generation", #task
    model=model_LLM,
    tokenizer=tokenizer,
    trust_remote_code=True,
    device_map="auto",
    torch_dtype=torch.bfloat16,
    max_length=1000,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,

)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


pytorch_model.bin.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

pytorch_model-00001-of-00003.bin:   0%|          | 0.00/9.88G [00:00<?, ?B/s]

pytorch_model-00002-of-00003.bin:   0%|          | 0.00/9.89G [00:00<?, ?B/s]

pytorch_model-00003-of-00003.bin:   0%|          | 0.00/7.18G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

In [15]:
llm = HuggingFacePipeline(pipeline = pipeline, model_kwargs = {'temperature':0})

In [35]:
template = """
A virtual assistant answers questions from a user based on the provided text.
USER: Text: {input_text}
ASSISTANT: I’ve read this text.
USER: What describes {entity_type} in the text?
ASSISTANT:
"""

prompt = PromptTemplate(template=template, input_variables=["input_text", "entity_type"])

llm_chain = LLMChain(prompt=prompt, llm=llm)

In [80]:
text = multimodal
entity_type = "person"

print(llm_chain.run({"input_text":text, "entity_type":entity_type}))

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(



A virtual assistant answers questions from a user based on the provided text.
USER: Text: This guy used to rule America in the 1920s a man in a suit is reading a book "Our greatest weakness lies in giving up: The most certain way to succeed is always to try just one more time:' Thomas Edison Forbes 
ASSISTANT: I’ve read this text.
USER: What describes person in the text?
ASSISTANT:
 ["Thomas Edison"]


In [81]:
text = multimodal
entity_type = "location"

print(llm_chain.run({"input_text":text, "entity_type":entity_type}))

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(



A virtual assistant answers questions from a user based on the provided text.
USER: Text: This guy used to rule America in the 1920s a man in a suit is reading a book "Our greatest weakness lies in giving up: The most certain way to succeed is always to try just one more time:' Thomas Edison Forbes 
ASSISTANT: I’ve read this text.
USER: What describes location in the text?
ASSISTANT:
 ["America"]


In [82]:
text = multimodal
entity_type = "motivation"

print(llm_chain.run({"input_text":text, "entity_type":entity_type}))

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(



A virtual assistant answers questions from a user based on the provided text.
USER: Text: This guy used to rule America in the 1920s a man in a suit is reading a book "Our greatest weakness lies in giving up: The most certain way to succeed is always to try just one more time:' Thomas Edison Forbes 
ASSISTANT: I’ve read this text.
USER: What describes motivation in the text?
ASSISTANT:
["certain way to succeed"]


# NER using spacy

In [ ]:
import re
import spacy
from spacy import displacy

In [ ]:
!python -m spacy download en_core_web_lg

  Using cached https://github.com/explosion/spacy-models/releases/download/en_core_web_lg-3.7.1/en_core_web_lg-3.7.1-py3-none-any.whl (587.7 MB)
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


DEPRECATION: pyraftlog 3.0.0 has a non-standard dependency specifier msgpack>="0.6.1". pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyraftlog or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
DEPRECATION: pyraftlog 3.0.0 has a non-standard dependency specifier redis>="3.0.0". pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyraftlog or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
ner = spacy.load("en_core_web_lg")
ner

In [ ]:
doc = ner(multimodal)

In [ ]:
doc.ents

(America, the 1920s, Thomas Edison)

In [ ]:
displacy.render(doc, style="dep", jupyter=True)

In [ ]:
displacy.render(doc, style="ent", jupyter=True)

In [ ]:
entities = [(ent.text, ent.label_) for ent in doc.ents]
entities

[('America', 'GPE'), ('the 1920s', 'DATE'), ('Thomas Edison', 'PERSON')]